In [2]:
import numpy as np
import pandas as pd

from litedl.models import FeedForwardNeuralNetwork
from litedl.layers import Affine, Sigmoid, ReLU, MSE
from litedl.optimizers import SGD


dataset = pd.read_csv('dataset/advertising.csv')
mean = dataset.mean().values
std = dataset.std().values
dataset = (dataset - mean) / std
features = dataset.iloc[:, :-1].values
labels = dataset.iloc[:, -1].values.reshape(-1, 1)
print(mean, std)

train_features = features[:160]
train_labels = labels[:160]

test_features = features[160:]
test_labels = labels[160:]

model = FeedForwardNeuralNetwork()
affine1 = Affine(input_size=3, output_size=50)
sigmoid = Sigmoid()
affine2 = Affine(input_size=3, output_size=1)
mse = MSE()
sgd = SGD()

# model.add_layer(affine1)
# model.add_layer(sigmoid)
model.add_layer(affine2)
model.add_loss_layer(mse)

for i in range(500):
    model.forward(train_features, train_labels)
    model.backward()
    model.step(optimizer=sgd)
    
    print(f'epoch {i}: {model.forward(train_features, train_labels)}')

print('=================================')
y_hat = (model.predict(test_features) * std[-1] + mean[-1]).flatten()
y = (test_labels * std[-1] + mean[-1]).flatten()

compare = pd.DataFrame({'Actual': y, 'Predicted': y_hat, 'Error': np.abs(y - y_hat)})
print(compare[compare['Error'] > 2])